<a href="https://colab.research.google.com/github/huyhoangttql/testML/blob/main/ML_20202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers import Flatten
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import GlobalMaxPooling1D
from keras.layers import Conv1D
from keras.layers import LSTM
from keras import *
df = pd.read_csv('/content/drive/MyDrive/Dataset/IMDB Dataset.csv')
#df['split'] = np.random.randn(df.shape[0],1)   # them cot split co gia tri theo pp chuan, df.shape[0] = 50000
# print (df)
# sns.countplot(x='sentiment', data=df)

'''
msk = np.random.rand(len(df)) <= 0.7   # msk = true neu so ngau nhien sinh ra <= 0.7
print(msk)
train = df[msk]
test = df[~msk]
print(train.shape)
print(len(train['sentiment']))
print(test)
'''
print(len(df['review'][0]))



1761


In [3]:
#Preprocessing
def preprocess_text(sen):
  # Xoa the html
  sentence = remove_tags(sen)

  # Xoa dau cau va so
  sentence = re.sub('[^a-zA-Z]', ' ', sentence)
 
  # Xoa ki tu don
  sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

  # Xoa nhieu space
  sentence = re.sub(r'\s+', ' ', sentence)

  return sentence

tag_re = re.compile(r'<[^>]+>')
def remove_tags(text):
  return tag_re.sub('', text) 

x = []  # x[i] chua 1 cmt da duoc tien xu ly
sentences = list(df['review'])
for sen in sentences:
  x.append(preprocess_text(sen))
#print (x[3])

# Chuyen doi label: positive->1, negative->0
y = df['sentiment']
y = np.array(list(map(lambda x: 1 if x=="positive" else 0, y)))   # y[i] chua label duoc chuyen dang so 0,1

# Chia dataset thanh training set and testing set
# x_train,x_test,y_train,y_test = train_test_split(x, y, test_size = 0.3, random_state = 42)
# print(x_train[1])
# print(y_train[1])

print(type(x))
print(type(y))
print(len(x[0]))
for sentence in x:
  if "this film was just brilliant casting location scenery" in sentence:
    print(x.index(sentence))
print(x[4788])
print(y[4788])

<class 'list'>
<class 'numpy.ndarray'>
1653
4788
this film was just brilliant casting location scenery story direction everyone really suited the part they played and you could just imagine being there Robert Redford is an amazing actor and now the same being director Norman father came from the same Scottish island as myself so loved the fact there was real connection with this film the witty remarks throughout the film were great it was just brilliant so much that bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at film it must have been good and this definitely was also congratulations to the two little boy that played the part of Norman and Paul they were just brilliant children are often left out of the praising list think because the stars that play them all grown up are such big profile for the whole film but these children 

In [4]:
# Prepareing embedding layer
# Su dung Tokennizer de tao tu dien word-to-index (moi tu duoc bieu dien la 1 so)
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(x)   # cap nhat tu dien dua tren tap x_train
# tokenizer.fit_on_texts(y)

x = tokenizer.texts_to_sequences(x)   # chuyen text thanh chuoi so nguyen
# y = tokenizer.texts_to_sequences(x_test)

vocab_size = len(tokenizer.word_index) + 1
maxlen =100  # Kich thuoc moi danh sach


# x = np.asarray(x)
print(x[4788])
# data = np.concatenate((x_train, testing_data), axis=0)

[8, 16, 10, 37, 502, 1027, 1435, 1279, 59, 454, 290, 60, 4016, 1, 171, 28, 249, 2, 17, 92, 37, 801, 105, 35, 596, 5, 30, 483, 275, 2, 144, 1, 166, 105, 151, 3184, 323, 377, 33, 1, 166, 4808, 901, 11, 525, 32, 427, 1, 190, 35, 10, 142, 1868, 14, 8, 16, 1, 1873, 466, 1, 16, 67, 76, 6, 10, 37, 502, 32, 69, 9, 1200, 1, 16, 11, 518, 11, 6, 10, 601, 13, 2, 56, 370, 6, 4, 290, 4, 99, 2, 1, 1991, 10, 483, 60, 3533, 27, 1, 125, 6, 10, 32, 608, 2, 17, 115, 43, 28, 130, 41, 17, 1400, 27, 16, 6, 200, 23, 72, 45, 2, 8, 402, 10, 78, 4, 1, 102, 116, 394, 9, 249, 1, 171, 3, 3184, 2, 803, 28, 67, 37, 502, 404, 21, 400, 308, 39, 3, 1, 987, 98, 80, 1, 398, 9, 282, 88, 26, 2022, 51, 21, 137, 186, 13, 1, 218, 16, 15, 131, 404, 21, 483, 2, 139, 25, 13, 43, 28, 23, 217, 82, 17, 98, 1, 218, 59, 10, 32, 1287, 80, 6, 10, 285, 2, 10, 278, 109, 96, 26, 9, 10, 14, 176, 26]


In [5]:
def vectorize(sequences, dimension = 10000):
  results = np.zeros((len(sequences), dimension))
  for i, sequence in enumerate(sequences):
    results[i, sequence] = 1
  return results
 
# data = vectorize(x)
print(type(x))
y=np.asarray(y).astype(np.float32)

<class 'list'>


In [6]:
test_x = x[:10000]
test_y = y[:10000]
train_x = x[10000:]
train_y = y[10000:]
print(type(test_y))
print(type(test_x))

<class 'numpy.ndarray'>
<class 'list'>


In [7]:
model = Sequential()
# Input - Layer
model.add(layers.Dense(50, activation = "relu", input_shape=(10000, )))
# Hidden - Layers
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
# Output- Layer
model.add(layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                500050    
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total params: 505,201
Trainable params: 505,201
Non-trainable params: 0
__________________________________________________

In [8]:
model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

In [9]:
train_x = vectorize(train_x)
test_x = vectorize(test_x)
train_y = np.asarray(train_y).astype('float32')
test_y = np.asarray(test_y).astype('float32')

In [1]:
results = model.fit(
 train_x, train_y,
 epochs= 2,
 batch_size = 1000,
 validation_data=(test_x,test_y),
)
print("Test-Accuracy:", np.mean(results.history["val_accuracy"]))

NameError: ignored